In [1]:
import statistics as stat
import pandas as pd
import numpy as np
import math
import json

In [2]:
# 0. Чтение данных из подготовленных файлов и объявление констант.
data = pd.read_csv("resources/data.csv", delimiter=', ').drop(columns=["Unnamed: 0"])
context_day = pd.read_csv("resources/context_day.csv", delimiter=', ').drop(columns=["Unnamed: 0"])
context_place = pd.read_csv("resources/context_place.csv", delimiter=', ').drop(columns=["Unnamed: 0"])

my_index = 11
non_rated = -1

home = 'h'
non_home = ['v', 'c']

weekends = ['Sat', 'Sun']
non_weekends = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']

def r3(x):
    return round(x, 3)

In [3]:
# 1. Для заданного пользователя (12 Ковшов Валерий) рассчитать оценки для всех фильмов, которые он не оценил.
my_data_row = data.iloc[[my_index]].values[0]
row_len = len(my_data_row)

sims = {}
for row_index in range(len(data)):
    if row_index == my_index:
        continue
    user_data_row = data.iloc[[row_index]].values[0]
    sum_ui_vi = 0
    sum_ui = 0
    sum_vi = 0
    for column_index in range(row_len):
        my_data_cell = my_data_row[column_index]
        user_data_cell = user_data_row[column_index]
        if (my_data_cell != non_rated) & (user_data_cell != non_rated):
            sum_ui_vi += my_data_cell * user_data_cell
            sum_ui += my_data_cell ** 2
            sum_vi += user_data_cell ** 2
    sim_u_v = r3(sum_ui_vi / (math.sqrt(sum_ui) * math.sqrt(sum_vi)))
    sims[row_index] = sim_u_v

sorted_sims = dict(sorted(sims.items(), key=lambda x: x[1], reverse=True))

avg_for_user = {}
for row_index in range(len(data)):
    avg = 0
    count = 0
    data_row = data.iloc[[row_index]].values[0]
    for column_index in range(row_len):
        data_cell = data_row[column_index]
        if data_cell != non_rated:
            avg += data_cell
            count += 1
    avg_for_user[row_index] = r3(avg / count)

my_non_rated = []
future_rate = {}
for movie_index in range(row_len):
    if my_data_row[movie_index] == non_rated:
        my_non_rated.append(movie_index)
        sum_numerator = 0
        sum_denominator = 0
        count = 0
        for user_index in sorted_sims:
            user_rate = data.iloc[[user_index]].values[0][movie_index]
            if user_rate == non_rated:
                count += 1
            elif count == 4:
                break
            else:
                count += 1
                sum_numerator += sims[user_index] * (user_rate - avg_for_user[user_index]) 
                sum_denominator += sims[user_index]
        future_rate[data.columns[movie_index]] = r3(avg_for_user[my_index] + sum_numerator / sum_denominator)
print("part 1 answer = ", future_rate)

part 1 answer =  {'Movie 4': 3.538, 'Movie 6': 3.92, 'Movie 11': 2.556, 'Movie 20': 3.591}


In [4]:
# 2. Порекомендовать заданному пользователю 1 фильм, который он посмотрел бы в выходной дома, если такой фильм есть в данных.
# Мы будем выбирать из фильмов, которые пользователь ещё не смотрел иначе какой смысл? Ориенироваться при выборе будем на оценки пользователей, которые смотрели эти фильмы (которые не видел заданный пользователь) дома в выходные с учётом соотношения вкусов этих пользователей с вкусами заданного пользователя.

avg_for_movie = {}
for column_index in range(len(my_non_rated)):
    avg = 0
    count = 0
    data_column = data[data.columns[my_non_rated[column_index]]]
    for row_index in range(len(data)):
        data_cell = data_column[row_index]
        if data_cell != non_rated:
            avg += data_cell
            count += 1
    avg_for_movie[my_non_rated[column_index]] = r3(avg / count)

context_day_for_non_rated = context_day[context_day.columns[my_non_rated]]
context_day_for_non_rated_at_weekends = context_day_for_non_rated.replace(weekends, 1).replace(non_weekends, 0)

context_place_for_non_rated = context_place[context_place.columns[my_non_rated]]
context_place_for_non_rated_at_home = context_place_for_non_rated.replace(home, 1).replace(non_home, 0)

weekends_at_home = {}
for user_index in range(len(data)):
    weekends_at_home[user_index] = []
    user_context_day = context_day_for_non_rated_at_weekends.iloc[[user_index]].values[0]
    user_context_place = context_place_for_non_rated_at_home.iloc[[user_index]].values[0]
    for movie_index in range(len(my_non_rated)):
        context_day_cell = int(user_context_day[movie_index])
        context_place_cell = int(user_context_place[movie_index])
        if context_day_cell == 1 & context_place_cell == 1:
            weekends_at_home[user_index].append(True)
        else:
            weekends_at_home[user_index].append(False)

rate_for_non_rated_at_weekends_at_home = {}
for movie_index in range(len(my_non_rated)):
    sum_numerator = 0
    sum_denominator = 0
    for user_index in weekends_at_home:
        if weekends_at_home[user_index][movie_index] == False:
            continue
        sum_numerator += sims[user_index] * (data.iloc[[user_index]].values[0][my_non_rated[movie_index]] - avg_for_movie[my_non_rated[movie_index]])
        sum_denominator += sims[user_index]
    rate_for_non_rated_at_weekends_at_home[data.columns[my_non_rated[movie_index]]] = avg_for_movie[my_non_rated[movie_index]] + r3(sum_numerator / sum_denominator)
top_movie_for_weekends_at_home_tuple = sorted(rate_for_non_rated_at_weekends_at_home.items(), key=lambda x: x[1], reverse=True)[0]
top_movie_for_weekends_at_home = {top_movie_for_weekends_at_home_tuple[0]: top_movie_for_weekends_at_home_tuple[1]}
print("part 2 answer = ", top_movie_for_weekends_at_home)

part 2 answer =  {'Movie 4': 3.199}


In [5]:
# 3. Заполнение json файла.
user = my_index + 1
ans_part1 = json.dumps(future_rate, sort_keys=True, indent=4)
ans_part2 = json.dumps(top_movie_for_weekends_at_home, sort_keys=True, indent=4) 
result = json.dumps({'user': user,'1': future_rate, '2': top_movie_for_weekends_at_home}, indent=4)
print(result)

{
    "user": 12,
    "1": {
        "Movie 4": 3.538,
        "Movie 6": 3.92,
        "Movie 11": 2.556,
        "Movie 20": 3.591
    },
    "2": {
        "Movie 4": 3.199
    }
}
